<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Dados_ALESP_Caio_%C3%8Dtalo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resumo
- Extração, Transformação e Carregamento de dados de Dataset da Assembleia Legislativa de São Paulo. Com tratamento de dados e união de tabelas.
- Breves análises de dados


# **EXTRAÇÃO**

In [1]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 122 kB 13.3 MB/s 


In [52]:
# Importação de bibliotecas
import pandas as pd
import pandera as pa

In [105]:
# dados alocados em um bucket na Google Cloud Platform para evitar indisponibilidade
df = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/alesp_desp.csv')
dfbackup = df.copy()
pd.set_option('display.max_rows', 50)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# **Transformação**

In [ ]:
df

## Renomeando e dropando colunas desnecessárias

In [108]:
df.rename(columns={'Ano':'ano','Matricula':'matricula','Mes':'mes','Valor':'valor','CNPJ':'cnpj','Tipo':'tipo', 'Fornecedor':'fornecedor', 'Deputado':'deputado'},inplace=True)
df.drop('Unnamed: 0',axis=1, inplace=True)

## Checagens básicas de inconsistências



In [68]:
# CNPJ como objeto é algo a ser observado.

df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj           object
deputado       object
tipo           object
fornecedor     object
dtype: object

In [ ]:
# Usando unique para ver os valores únicos em cada coluna.
# pontos a observar: valores muito baixos como: 0.0; 0.1; 0.2; 0.3
# Muitos fornecedores com nomes parecidos

sorted(pd.unique(df['fornecedor']))

In [18]:
# Checagem de valores NaN
df.isna().sum()

ano               0
matricula         0
mes               0
valor             0
cnpj          10892
deputado          0
tipo              0
fornecedor        0
dtype: int64

## TRATAMENTO BÁSICO DA BD

In [ ]:
sorted(pd.unique(df['fornecedor']))

In [29]:
df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj           object
deputado       object
tipo           object
fornecedor     object
dtype: object

### O problema dos CNPJs

In [30]:
# verificando se todos os CNPJs são numéricos, percebe-se que não.
df.cnpj.astype(float)

ValueError: ignored

In [84]:
# Coletando os valores únicos de CNPJ
dfun = df.cnpj.unique()
dfun

array(['71806251000106', '68064740000125', '22145388877', ...,
       50323741991.0, 6878968000100.0, 5591224000139.0], dtype=object)

In [109]:
# Percorrendo os valores únicos para encontrar os cnpjs que não são númericos
valores_estranhos = []
for i in range(len(dfun)):
  try:
    int(dfun[i])
  except Exception:
    valores_estranhos.append(dfun[i])
print(valores_estranhos)

[nan, 'O4689799000126', 'O4040553000129', 'O2358317000120', 'O1520430000105', '00O21449000136', 'O7482309000104']


In [110]:
# todos os casos em que uma letra 'O' foi colocado em vez do 0
ftcnpj = df[df.cnpj.isin(['O4689799000126','O4040553000129','O2358317000120','O1520430000105','00O21449000136','O7482309000104'])]
ftcnpj

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
55891,2007,300482,11,76.78,O4689799000126,ANTÔNIO CARLOS,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",VIA BARBARESCO LTDA
55964,2008,300482,2,70.02,O4040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
55998,2008,300482,3,50.00,O4040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
316251,2007,300455,9,40.00,O2358317000120,MARCOS MARTINS,"G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, IN...",BASENET INFORMATICA LTDA
365639,2007,300468,12,504.12,O1520430000105,OLÍMPIO GOMES,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PICANHARIA DO GAUCHO/ JOCELI NARDI CHURRASCARI...
389510,2002,300379,5,786.55,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389553,2002,300379,9,915.76,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389565,2002,300379,10,802.69,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389578,2002,300379,12,3101.63,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389582,2003,300379,1,5472.30,00O21449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA


In [111]:
# Trocando o erro, muito provavelmente por digitações erradas em sistemas antigos
ftcnpj.cnpj.replace('O','0', regex=True, inplace=True)
ftcnpj

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
55891,2007,300482,11,76.78,04689799000126,ANTÔNIO CARLOS,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",VIA BARBARESCO LTDA
55964,2008,300482,2,70.02,04040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
55998,2008,300482,3,50.00,04040553000129,ANTÔNIO CARLOS,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO AUSTRIA LTDA
316251,2007,300455,9,40.00,02358317000120,MARCOS MARTINS,"G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, IN...",BASENET INFORMATICA LTDA
365639,2007,300468,12,504.12,01520430000105,OLÍMPIO GOMES,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PICANHARIA DO GAUCHO/ JOCELI NARDI CHURRASCARI...
389510,2002,300379,5,786.55,00021449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389553,2002,300379,9,915.76,00021449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389565,2002,300379,10,802.69,00021449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389578,2002,300379,12,3101.63,00021449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA
389582,2003,300379,1,5472.30,00021449000136,PEDRO YVES,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO BOSQUE SATÉLITE LTDA


In [130]:
# Agrupando e mostrando os casos em que há valores vazios para possíveis inconsistências.
# Nota-se vários campos com nomes similares
dfcnpjnan = df[df.cnpj.isna()]

fornecedor
ADOBE SYSTEMS SOFTWARE IRELAND LTD                   1
ADOBE SYSTEMS SOFTWARE IRELAND LTDA                  5
CONCESSIONARIA RODOVIA OESTE DE SAO PAULO S.A.       1
CUPONS DE PEDÁGIO                                    1
DELTA RADIO TAXI                                     1
DROPBOX INTERNACIONAL UNLIMETED COMPANY              1
ELOY DE PAULA JUNIOR                                 1
FLICKR, INC                                          1
GOOGLE ONE                                           2
GOOGLE STORAGE - GOOGLE ONE                          1
LEANDRO MATOS NICOLELA                               1
PEDAGIO                                             11
PEDAGIOS                                             1
PEDÁGIO                                           7476
PEDÁGIOS                                             2
PEGAGIO                                              1
TAXI                                              3385
dtype: int64

In [133]:
dfcnpjnan.groupby(['fornecedor']).size()

fornecedor
ADOBE SYSTEMS SOFTWARE IRELAND LTDA                  6
CONCESSIONARIA RODOVIA OESTE DE SAO PAULO S.A.       1
DELTA RADIO TAXI                                     1
DROPBOX INTERNACIONAL UNLIMETED COMPANY              1
ELOY DE PAULA JUNIOR                                 1
FLICKR, INC                                          1
GOOGLE ONE                                           3
LEANDRO MATOS NICOLELA                               1
PEDÁGIO                                           7492
TAXI                                              3385
dtype: int64

In [ ]:
# Fazendo o replace dessas inconsistências
dfcnpjnan.fornecedor.replace({'ADOBE SYSTEMS SOFTWARE IRELAND LTD':'ADOBE SYSTEMS SOFTWARE IRELAND LTDA',
                              'PEDAGIOS':'PEDÁGIO','PEDAGIO':'PEDÁGIO','PEDÁGIOS':'PEDÁGIO','PEGAGIO':'PEDÁGIO','CUPONS DE PEDÁGIO':'PEDÁGIO',
                               'GOOGLE STORAGE - GOOGLE ONE':'GOOGLE ONE'},inplace=True)


In [27]:
# Criando um schema de validação de dados
schema = pa.DataFrameSchema(
  columns={
    'ano':pa.Column(pa.Int),
    'matricula':pa.Column(pa.Int),
    'mes':pa.Column(pa.Int),
    'valor':pa.Column(pa.Float),
    'cnpj':pa.Column(pa.String,nullable=True),
    'tipo':pa.Column(pa.String),
    'fornecedor':pa.Column(pa.String),
    'deputado':pa.Column(pa.String)
    }
)

In [28]:
# validando o DF
schema.validate(df)

,ano,matricula,mes,valor,cnpj,deputado,tipo,fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
536563,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
536564,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
536565,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
536566,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [ ]:
# pegando os dados da última legislatura
dfleg = df[(df.ano > 2018) & (df.ano < 2023)]
dfleg

In [ ]:
# # Verificador de letras em numeros
# valores_estranhos = []
# for i in range(len(df_despesas_cnpj)):
#   try:
#     int(df_despesas_cnpj.loc[i , 'CNPJ'])
#   except Exception:
#     valores_estranhos.append(df_despesas_cnpj.loc[i , 'CNPJ'])
# print(valores_estranhos)

#**CARREGAMENTO DOS DADOS**

#**ANÁLISES BÁSICAS DOS DADOS**